<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Библиотеки-для-работы" data-toc-modified-id="Библиотеки-для-работы-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Библиотеки для работы</a></span></li><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Итоги-подбора" data-toc-modified-id="Итоги-подбора-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Итоги подбора</a></span></li></ul></li><li><span><a href="#Тест-лучшей-модели" data-toc-modified-id="Тест-лучшей-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тест лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

**Код подготовлен для запуска с использованием CUDA. Должен сработать без проблем :-)**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**Целевое значение метрики качества *F1* - не меньше 0.75**

Для выполнения проекта было решено применить модель *BERT*. 

**Важно:** ресурсов локальной машины было недостаточно для адекватной по времени работы, поэтому часть кода, формирующей эмбендинги была запущена на компьютере однокурсника по Практикуму. Надеюсь на понимание ревьюера :-)

Ссылка на файл с эмбендингами, полученными на удаленном компьютере и использованными для обучения моделей: https://disk.yandex.ru/d/ZYc6VzeMnf4wxQ

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Библиотеки для работы

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

D:\anaconda3\envs\test_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Проверим, доступны ли нам расчёты на видеокарте

In [2]:
torch.cuda.is_available()

False

In [3]:
print(torch.version.cuda)

None


К сожалению, на личной машине эта возможность отсутствует. Но оочень нравится BERT своей точностью, поэтому код будем писать всё же с применением технологии CUDA и попросим сформировать эмбендинги владельца более мощного ПК.

## Знакомство с данными

Загружаем данные и смотрим общую информацию получившегося датасета

In [3]:
try:
    comments = pd.read_csv('toxic_comments.csv')
except:
    comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
    
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Всегда б так было - пропусков нет, датасет на первый взгляд идеален. Взглянем визуально на случайные 5 строк датасета

In [4]:
comments.sample(5)

,Unnamed: 0,text,toxic
88855,88938,"""\n\nThe right template\n\nThanks for showing ...",0
118487,118587,"""\n\n Yamla the idiot \n\nIf (user.IQ<60)\n{\n...",1
93257,93349,"Disambigation colour \n\nHi, I don't believe I...",0
107477,107574,The American Indian Movement\n\nRemember Wound...,0
107489,107586,I've made a request according to the instructi...,0


А вот тут сюрприз - комментарии на английском языке. Но ничего, подберем подходящую модель BERT, которая всё поймёт.

## Подготовка данных

Используем AutoModel и AutoTokenizer из библиотеки transformers, а веса подгрузим из модели `toxic-bert`

In [5]:
model_class, tokenizer_class, pretrained_weights = (
    transformers.AutoModel, transformers.AutoTokenizer, "unitary/toxic-bert")

Проведем токинезацию и подготовим маску внимания

In [6]:
%%time
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

#Токенизируем наш текст и заменяем каждый токен на уникальый идентификатор 
tokenized = comments['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True))

#Дополняем каждую строку нулями до тех пор , пока не достигнем максимальной длины одной из строк
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array(
    [i + [0]*(max_len - len(i)) for i in tokenized.values])

# Создаём маску "внимания", для того чтобы игнорировать добавленные нули
attention_mask = np.where(padded != 0, 1, 0)

CPU times: total: 1min 48s
Wall time: 1min 56s


Готово. Проверим размерности получившихся массивов

In [7]:
print(padded.shape)
print(attention_mask.shape)

(159292, 512)
(159292, 512)


Размерности совпадают, двигаемся дальше - задаём модель с весами заранее тренированной модели BERT

In [8]:
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Помещаем модель в ?память? видеокарты для кратного увеличения скорости расчётов

In [14]:
# ячейка для запуска не на личной машине
model = model.cuda()

Формируем эмбеддинги, обращаем внимание, что в ходе работы тензоры должны формироваться в ?памяти? видеокарты, но результат предполагаю сделать доступным для обработки процессором. Поэтому в методе `append` добавил `cpu()`

In [15]:
# ячейка для запуска не на личной машине
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cuda().cpu())

  0%|          | 0/1592 [00:00<?, ?it/s]

Отлично, прогрессбар радует 100% выполнения. Сформируем массив со свойствами и целевым признаком. Проверим размерность. 

In [10]:
features = np.concatenate(embeddings)
target = comments['toxic']

# np.save('features', features)  
# данная строчка использовалась для выгрузки признаков в файл и последующего его использования на
# личном ПК

#features = np.load('features.npy')
# данная строчка использовалась для подгрузки сформированных на чужом ПК эмбендингов 
# на локальную машину

print(features.shape)
target.shape

(159200, 768)


(159292,)

Похоже последний батч не дотянулся количеством объектов и не был обработан. Так как батчи формировались по порядку, просто обрежем массив снизу, оставив необходимые 159 200 объектов. И снова проверим размерность

In [11]:
target = comments['toxic'].head(159200)

print(features.shape)
target.shape

(159200, 768)


(159200,)

Порядок.

## Обучение

 Приступаем к формированию выборок и подбору параметров моделей. Тестовая выборка в 30% от общей.

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=12345)

Напишем функцию для подбора параметров модели для лучшей метрики F1

In [13]:
def best_model(model, hyperparams):
    # готовим и обучаем классификатор
    clf = GridSearchCV(model, hyperparams, scoring='f1', verbose=2)
    print('Начат подбор параметров:')
    clf.fit(features_train, target_train)
    
    # получаем результаты подбора и записываем в переменные время обучения и предсказания
    results = pd.DataFrame(clf.cv_results_)
    fit_time = results[results['rank_test_score'] == 1]['mean_fit_time'].values[0]
    predict_time = results[results['rank_test_score'] == 1]['mean_score_time'].values[0]
    
    # записываем в переменные лучшую модель, её параметры и RMSE
    model = clf.best_estimator_
    params = clf.best_params_
    f1 = clf.best_score_   
    
    # выводим результат
    print(f'Лучшая модель {model}')
    print(params)
    print()
    print(f'Показатель F1: {f1}')
    print()
    print(f'Время обучения: {fit_time}; Время предсказания: {predict_time}')
    return model, params, f1, fit_time, predict_time

### LogisticRegression

Логистическую регрессию проверим без GridSearchCV, т.к. подбирать особо нечего.

In [15]:
model_lr = LogisticRegression(max_iter=1000, class_weight='balanced')
model_lr.fit(features_train, target_train)

lr_f1 = cross_val_score(model_lr, features_test, target_test, scoring='f1')
print(f'Показатель F1: {lr_f1.mean()}')

D:\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Показатель F1: 0.9349033785550775


Хороший показатель! Чем ответят другие модели?

### DecisionTreeClassifier

Поработаем с решающим деревом с отбалансированными классами. Тяжеловато моему ПК приходится, поэтому буду перебирать всего парочку глубин деревьев.

In [16]:
model_dtc = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
hyperparams = {'max_depth':[5, 10, 15]}

model_dtc, dtc_params, dtc_f1, dtc_fit_time, dtc_predict_time = best_model(model_dtc, hyperparams)

Начат подбор параметров:
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ........................................max_depth=5; total time= 1.1min
[CV] END ........................................max_depth=5; total time= 1.1min
[CV] END ........................................max_depth=5; total time= 1.1min
[CV] END ........................................max_depth=5; total time= 1.1min
[CV] END ........................................max_depth=5; total time= 1.1min
[CV] END .......................................max_depth=10; total time= 2.2min
[CV] END .......................................max_depth=10; total time= 2.2min
[CV] END .......................................max_depth=10; total time= 2.1min
[CV] END .......................................max_depth=10; total time= 2.1min
[CV] END .......................................max_depth=10; total time= 2.1min
[CV] END .......................................max_depth=15; total time= 3.2min
[CV] END ...............

Логистическая регрессия осталась на вершине точности.

### CatBoostClassifier

CatBoost понравился в предыдущих проектах, поэтому попробуем классификатор от Яндекса и здесь. Правда придётся предварительно отбалансировать классы.

In [17]:
# считываем уникальные классы
classes = np.unique(target_train)
# применяем специальную функцию из sklearn для формирования весов классов
weights = compute_class_weight(class_weight='balanced', classes=classes, y=target_train)
# формируем словарь с классами и их весами
class_weights = dict(zip(classes, weights))

class_weights

{0: 0.5565154859522787, 1: 4.923566316161527}

Всё прошло удачно. Подберем параметры для CatBoost (уменьшил количество итераций для ускорения процесса)

In [18]:
model_cbc = CatBoostClassifier(random_state=12345, iterations=500, class_weights=class_weights) 
hyperparams = {'learning_rate':[0.1, 0.8]}

model_cbc, cbc_params, cbc_rmse, cbc_fit_time, cbc_predict_time = best_model(model_cbc, hyperparams)

Начат подбор параметров:
Fitting 5 folds for each of 2 candidates, totalling 10 fits
0:	learn: 0.4892248	total: 323ms	remaining: 2m 41s
1:	learn: 0.3472605	total: 512ms	remaining: 2m 7s
2:	learn: 0.2501107	total: 704ms	remaining: 1m 56s
3:	learn: 0.1945637	total: 899ms	remaining: 1m 51s
4:	learn: 0.1512579	total: 1.09s	remaining: 1m 47s
5:	learn: 0.1251504	total: 1.28s	remaining: 1m 45s
6:	learn: 0.1030352	total: 1.47s	remaining: 1m 43s
7:	learn: 0.0886705	total: 1.66s	remaining: 1m 42s
8:	learn: 0.0795046	total: 1.85s	remaining: 1m 41s
9:	learn: 0.0732708	total: 2.04s	remaining: 1m 40s
10:	learn: 0.0688739	total: 2.25s	remaining: 1m 40s
11:	learn: 0.0651855	total: 2.44s	remaining: 1m 39s
12:	learn: 0.0624809	total: 2.64s	remaining: 1m 38s
13:	learn: 0.0603405	total: 2.82s	remaining: 1m 38s
14:	learn: 0.0582978	total: 3.02s	remaining: 1m 37s
15:	learn: 0.0567427	total: 3.21s	remaining: 1m 37s
16:	learn: 0.0552931	total: 3.4s	remaining: 1m 36s
17:	learn: 0.0544380	total: 3.57s	remaining

156:	learn: 0.0283358	total: 29.9s	remaining: 1m 5s
157:	learn: 0.0282638	total: 30.1s	remaining: 1m 5s
158:	learn: 0.0282183	total: 30.3s	remaining: 1m 5s
159:	learn: 0.0281175	total: 30.5s	remaining: 1m 4s
160:	learn: 0.0280101	total: 30.7s	remaining: 1m 4s
161:	learn: 0.0278872	total: 30.9s	remaining: 1m 4s
162:	learn: 0.0278137	total: 31.1s	remaining: 1m 4s
163:	learn: 0.0277351	total: 31.3s	remaining: 1m 4s
164:	learn: 0.0276443	total: 31.5s	remaining: 1m 3s
165:	learn: 0.0275890	total: 31.7s	remaining: 1m 3s
166:	learn: 0.0275104	total: 31.9s	remaining: 1m 3s
167:	learn: 0.0274229	total: 32.1s	remaining: 1m 3s
168:	learn: 0.0273145	total: 32.2s	remaining: 1m 3s
169:	learn: 0.0272084	total: 32.4s	remaining: 1m 2s
170:	learn: 0.0271123	total: 32.6s	remaining: 1m 2s
171:	learn: 0.0269983	total: 32.8s	remaining: 1m 2s
172:	learn: 0.0269324	total: 33s	remaining: 1m 2s
173:	learn: 0.0268761	total: 33.2s	remaining: 1m 2s
174:	learn: 0.0268046	total: 33.4s	remaining: 1m 2s
175:	learn: 0.

316:	learn: 0.0190498	total: 1m	remaining: 34.7s
317:	learn: 0.0189961	total: 1m	remaining: 34.5s
318:	learn: 0.0189513	total: 1m	remaining: 34.3s
319:	learn: 0.0189217	total: 1m	remaining: 34.1s
320:	learn: 0.0188866	total: 1m	remaining: 33.9s
321:	learn: 0.0188377	total: 1m 1s	remaining: 33.7s
322:	learn: 0.0188030	total: 1m 1s	remaining: 33.5s
323:	learn: 0.0187800	total: 1m 1s	remaining: 33.4s
324:	learn: 0.0187383	total: 1m 1s	remaining: 33.2s
325:	learn: 0.0187071	total: 1m 1s	remaining: 33s
326:	learn: 0.0186719	total: 1m 1s	remaining: 32.8s
327:	learn: 0.0186478	total: 1m 2s	remaining: 32.6s
328:	learn: 0.0186228	total: 1m 2s	remaining: 32.4s
329:	learn: 0.0185647	total: 1m 2s	remaining: 32.2s
330:	learn: 0.0185442	total: 1m 2s	remaining: 32s
331:	learn: 0.0184823	total: 1m 2s	remaining: 31.8s
332:	learn: 0.0184381	total: 1m 3s	remaining: 31.7s
333:	learn: 0.0183797	total: 1m 3s	remaining: 31.5s
334:	learn: 0.0183446	total: 1m 3s	remaining: 31.3s
335:	learn: 0.0183041	total: 1m

473:	learn: 0.0150007	total: 1m 29s	remaining: 4.93s
474:	learn: 0.0150007	total: 1m 30s	remaining: 4.75s
475:	learn: 0.0150007	total: 1m 30s	remaining: 4.56s
476:	learn: 0.0150007	total: 1m 30s	remaining: 4.37s
477:	learn: 0.0150007	total: 1m 30s	remaining: 4.18s
478:	learn: 0.0150007	total: 1m 30s	remaining: 3.99s
479:	learn: 0.0150007	total: 1m 31s	remaining: 3.8s
480:	learn: 0.0150007	total: 1m 31s	remaining: 3.61s
481:	learn: 0.0150007	total: 1m 31s	remaining: 3.42s
482:	learn: 0.0150007	total: 1m 31s	remaining: 3.23s
483:	learn: 0.0150007	total: 1m 32s	remaining: 3.04s
484:	learn: 0.0150007	total: 1m 32s	remaining: 2.85s
485:	learn: 0.0150007	total: 1m 32s	remaining: 2.66s
486:	learn: 0.0150007	total: 1m 32s	remaining: 2.47s
487:	learn: 0.0150007	total: 1m 32s	remaining: 2.28s
488:	learn: 0.0150007	total: 1m 33s	remaining: 2.09s
489:	learn: 0.0150007	total: 1m 33s	remaining: 1.9s
490:	learn: 0.0150007	total: 1m 33s	remaining: 1.71s
491:	learn: 0.0150007	total: 1m 33s	remaining: 1

130:	learn: 0.0309923	total: 25.1s	remaining: 1m 10s
131:	learn: 0.0308642	total: 25.3s	remaining: 1m 10s
132:	learn: 0.0307229	total: 25.5s	remaining: 1m 10s
133:	learn: 0.0306154	total: 25.7s	remaining: 1m 10s
134:	learn: 0.0304894	total: 25.9s	remaining: 1m 10s
135:	learn: 0.0303845	total: 26.1s	remaining: 1m 9s
136:	learn: 0.0302897	total: 26.3s	remaining: 1m 9s
137:	learn: 0.0301995	total: 26.5s	remaining: 1m 9s
138:	learn: 0.0300710	total: 26.8s	remaining: 1m 9s
139:	learn: 0.0299747	total: 26.9s	remaining: 1m 9s
140:	learn: 0.0297545	total: 27.1s	remaining: 1m 9s
141:	learn: 0.0296259	total: 27.3s	remaining: 1m 8s
142:	learn: 0.0295296	total: 27.5s	remaining: 1m 8s
143:	learn: 0.0294793	total: 27.7s	remaining: 1m 8s
144:	learn: 0.0293581	total: 27.9s	remaining: 1m 8s
145:	learn: 0.0292753	total: 28.1s	remaining: 1m 8s
146:	learn: 0.0291749	total: 28.3s	remaining: 1m 8s
147:	learn: 0.0291032	total: 28.5s	remaining: 1m 7s
148:	learn: 0.0289690	total: 28.7s	remaining: 1m 7s
149:	le

...

242:	learn: 0.0080475	total: 43.3s	remaining: 45.7s
243:	learn: 0.0080475	total: 43.4s	remaining: 45.6s
244:	learn: 0.0080475	total: 43.6s	remaining: 45.4s
245:	learn: 0.0080475	total: 43.8s	remaining: 45.2s
246:	learn: 0.0080475	total: 43.9s	remaining: 45s
247:	learn: 0.0080475	total: 44.1s	remaining: 44.8s
248:	learn: 0.0080475	total: 44.3s	remaining: 44.6s
249:	learn: 0.0080475	total: 44.4s	remaining: 44.4s
250:	learn: 0.0080475	total: 44.6s	remaining: 44.2s
251:	learn: 0.0080475	total: 44.8s	remaining: 44.1s
252:	learn: 0.0080475	total: 45s	remaining: 43.9s
253:	learn: 0.0080475	total: 45.1s	remaining: 43.7s
254:	learn: 0.0080475	total: 45.3s	remaining: 43.5s
255:	learn: 0.0080475	total: 45.5s	remaining: 43.3s
256:	learn: 0.0080475	total: 45.6s	remaining: 43.2s
257:	learn: 0.0080475	total: 45.8s	remaining: 43s
258:	learn: 0.0080475	total: 46s	remaining: 42.8s
259:	learn: 0.0080475	total: 46.2s	remaining: 42.6s
260:	learn: 0.0080475	total: 46.3s	remaining: 42.4s
261:	learn: 0.008047

402:	learn: 0.0080475	total: 1m 10s	remaining: 17.1s
403:	learn: 0.0080475	total: 1m 11s	remaining: 16.9s
404:	learn: 0.0080475	total: 1m 11s	remaining: 16.7s
405:	learn: 0.0080475	total: 1m 11s	remaining: 16.5s
406:	learn: 0.0080475	total: 1m 11s	remaining: 16.3s
407:	learn: 0.0080475	total: 1m 11s	remaining: 16.2s
408:	learn: 0.0080475	total: 1m 11s	remaining: 16s
409:	learn: 0.0080475	total: 1m 12s	remaining: 15.8s
410:	learn: 0.0080475	total: 1m 12s	remaining: 15.6s
411:	learn: 0.0080475	total: 1m 12s	remaining: 15.5s
412:	learn: 0.0080475	total: 1m 12s	remaining: 15.3s
413:	learn: 0.0080475	total: 1m 12s	remaining: 15.1s
414:	learn: 0.0080475	total: 1m 12s	remaining: 14.9s
415:	learn: 0.0080475	total: 1m 13s	remaining: 14.7s
416:	learn: 0.0080475	total: 1m 13s	remaining: 14.6s
417:	learn: 0.0080475	total: 1m 13s	remaining: 14.4s
418:	learn: 0.0080475	total: 1m 13s	remaining: 14.2s
419:	learn: 0.0080475	total: 1m 13s	remaining: 14s
420:	learn: 0.0080475	total: 1m 13s	remaining: 13.

58:	learn: 0.0424756	total: 12.4s	remaining: 1m 32s
59:	learn: 0.0422950	total: 12.6s	remaining: 1m 32s
60:	learn: 0.0420958	total: 12.8s	remaining: 1m 32s
61:	learn: 0.0419471	total: 13s	remaining: 1m 31s
62:	learn: 0.0418173	total: 13.2s	remaining: 1m 31s
63:	learn: 0.0415314	total: 13.4s	remaining: 1m 31s
64:	learn: 0.0413126	total: 13.6s	remaining: 1m 31s
65:	learn: 0.0411852	total: 13.8s	remaining: 1m 30s
66:	learn: 0.0410005	total: 14s	remaining: 1m 30s
67:	learn: 0.0408579	total: 14.2s	remaining: 1m 30s
68:	learn: 0.0407305	total: 14.4s	remaining: 1m 29s
69:	learn: 0.0405237	total: 14.6s	remaining: 1m 29s
70:	learn: 0.0403741	total: 14.8s	remaining: 1m 29s
71:	learn: 0.0402462	total: 15s	remaining: 1m 29s
72:	learn: 0.0401358	total: 15.2s	remaining: 1m 29s
73:	learn: 0.0399742	total: 15.4s	remaining: 1m 28s
74:	learn: 0.0398226	total: 15.6s	remaining: 1m 28s
75:	learn: 0.0396799	total: 15.9s	remaining: 1m 28s
76:	learn: 0.0395600	total: 16.1s	remaining: 1m 28s
77:	learn: 0.03939

216:	learn: 0.0263494	total: 44.9s	remaining: 58.6s
217:	learn: 0.0262818	total: 45.1s	remaining: 58.4s
218:	learn: 0.0262240	total: 45.3s	remaining: 58.1s
219:	learn: 0.0261790	total: 45.5s	remaining: 58s
220:	learn: 0.0261415	total: 45.7s	remaining: 57.7s
221:	learn: 0.0260692	total: 45.9s	remaining: 57.5s
222:	learn: 0.0260208	total: 46.1s	remaining: 57.3s
223:	learn: 0.0259731	total: 46.3s	remaining: 57.1s
224:	learn: 0.0259405	total: 46.5s	remaining: 56.9s
225:	learn: 0.0258798	total: 46.7s	remaining: 56.6s
226:	learn: 0.0258214	total: 46.9s	remaining: 56.4s
227:	learn: 0.0257691	total: 47.2s	remaining: 56.3s
228:	learn: 0.0257182	total: 47.4s	remaining: 56s
229:	learn: 0.0256511	total: 47.6s	remaining: 55.8s
230:	learn: 0.0255913	total: 47.8s	remaining: 55.6s
231:	learn: 0.0255278	total: 48s	remaining: 55.4s
232:	learn: 0.0254864	total: 48.2s	remaining: 55.2s
233:	learn: 0.0254449	total: 48.4s	remaining: 55s
234:	learn: 0.0253961	total: 48.6s	remaining: 54.8s
235:	learn: 0.025362

375:	learn: 0.0192112	total: 1m 17s	remaining: 25.4s
376:	learn: 0.0191712	total: 1m 17s	remaining: 25.2s
377:	learn: 0.0191051	total: 1m 17s	remaining: 25s
378:	learn: 0.0190851	total: 1m 17s	remaining: 24.8s
379:	learn: 0.0190279	total: 1m 17s	remaining: 24.6s
380:	learn: 0.0189987	total: 1m 18s	remaining: 24.4s
381:	learn: 0.0189575	total: 1m 18s	remaining: 24.2s
382:	learn: 0.0189314	total: 1m 18s	remaining: 24s
383:	learn: 0.0188828	total: 1m 18s	remaining: 23.8s
384:	learn: 0.0188435	total: 1m 18s	remaining: 23.6s
385:	learn: 0.0188163	total: 1m 19s	remaining: 23.4s
386:	learn: 0.0187849	total: 1m 19s	remaining: 23.2s
387:	learn: 0.0187700	total: 1m 19s	remaining: 23s
388:	learn: 0.0187516	total: 1m 19s	remaining: 22.8s
389:	learn: 0.0187108	total: 1m 19s	remaining: 22.6s
390:	learn: 0.0186891	total: 1m 20s	remaining: 22.4s
391:	learn: 0.0186625	total: 1m 20s	remaining: 22.1s
392:	learn: 0.0186493	total: 1m 20s	remaining: 21.9s
393:	learn: 0.0186340	total: 1m 20s	remaining: 21.7s

### Итоги подбора

Сформируем таблицу с результатами подбора параметров по трем моделям

In [23]:
model_choose = pd.DataFrame({'Модель':['LogisticRegression', 'DecisionTreeClassifier', 'CatBoostClassifier'],
                             'F1':[lr_f1.mean(), dtc_f1, cbc_rmse],
                             'Время обучения':['-', dtc_fit_time, cbc_fit_time],
                             'Время предсказания':['-', dtc_predict_time, cbc_predict_time]})

display(model_choose)

,Модель,F1,Время обучения,Время предсказания
0,LogisticRegression,0.934903,-,-
1,DecisionTreeClassifier,0.919132,185.394418,0.041528
2,CatBoostClassifier,0.934105,98.687559,0.725403


- Лучшая по точности модель - LogisticRegression
- Худшая по точности модель - DecisionTreeClassifier

При этом CatBoost отстал от лидера всего на 0,0008... Обидно, но результат на табло.

Если сравнивать Решающее дерево с Котом по времени работы, то CatBoost в два раза быстрее обучился и в 18 раз медленнее предсказывал.

## Тест лучшей модели

Проверим лучшую модель на тестовой выборке

In [24]:
prediction = model_lr.predict(features_test)
f1_test = f1_score(target_test, prediction)

print(f'F1 лучшей модели на тесте: {f1_test}')

F1 лучшей модели на тесте: 0.9286129970902037


## Выводы

Для работы над проектом был получен датасет не требующий классической предобработки, связанной с заполнением пропусков. Из этой базы данных был взят столбец с комментариями, которые вдальнейшем были преобразованы в эмбендинги с помощью AutoModel и AutoTokenizer из библиотеки transformers, а также весами подгруженными из модели toxic-bert.

После этого были предварительно обучены и оценены по точности модели LogisticRegression, DecisionTreeClassifier, CatBoostClassifier. В итоге этого процесса получены следующие выводы:

- Лучшая по точности модель - LogisticRegression
- Худшая по точности модель - DecisionTreeClassifier

При этом CatBoost отстал от лидера всего на 0,0008

Далее LogisticRegression была проверена на тестовой выборке с результатом по метрике **f1 в 0,93**. Что сильно выше обозначенной нами цели. Бейзлайн побит.